In [1]:
from __future__ import print_function

import os

import numpy as np
from keras import backend as K
from keras import layers
from keras.callbacks import ModelCheckpoint, CSVLogger
from keras.layers import Activation
from keras.layers import Input, Conv2D, ZeroPadding2D, MaxPooling2D, UpSampling2D, concatenate, Cropping2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.optimizers import Adam
from skimage.io import imsave

#from data import load_train_data, load_test_data

K.set_image_data_format('channels_last')  # TF dimension ordering in this code

img_rows = 720
img_cols = 960

smooth = 1.
epochs = 200



Using TensorFlow backend.


In [2]:
def identity_block(input_tensor, kernel_size, filters, stage, block):
    filters1, filters2, filters3 = filters
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
       
    x = Conv2D(filters1, (1, 1), name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, padding='same', name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    x = layers.add([x, input_tensor])
    x = Activation('relu')(x)
    return x










In [3]:
def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):
    filters1, filters2, filters3 = filters
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), strides=strides, name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, padding='same', name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    shortcut = Conv2D(filters3, (1, 1), strides=strides, name=conv_name_base + '1')(input_tensor)
    shortcut = BatchNormalization(axis=bn_axis, name=bn_name_base + '1')(shortcut)

    x = layers.add([x, shortcut])
    x = Activation('relu')(x)
    return x

In [4]:
def up_conv_block(input_tensor, kernel_size, filters, stage, block, strides=(1, 1)):
    filters1, filters2, filters3 = filters
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
    up_conv_name_base = 'up' + str(stage) + block + '_branch'
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = UpSampling2D(size=(2, 2), name=up_conv_name_base + '2a')(input_tensor)

    x = Conv2D(filters1, (1, 1), strides=strides, name=conv_name_base + '2a')(x)

    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, padding='same', name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    shortcut = UpSampling2D(size=(2, 2), name=up_conv_name_base + '1')(input_tensor)
    shortcut = Conv2D(filters3, (1, 1), strides=strides, name=conv_name_base + '1')(shortcut)
    shortcut = BatchNormalization(axis=bn_axis, name=bn_name_base + '1')(shortcut)

    x = layers.add([x, shortcut])
    x = Activation('relu')(x)
    return x

In [30]:
def get_resnet(f=16, bn_axis=3, classes=3):
    inp = Input((img_rows, img_cols, 3))
    x = Cropping2D(cropping=((8, 8), (0, 0)))(inp)
    x = ZeroPadding2D((4, 4))(x)
    x = Conv2D(f, (7, 7), strides=(2, 2), name='conv1')(x)
    x = BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv_block(x, 3, [f, f, f * 2], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [f, f, f * 2], stage=2, block='b')
    x2 = identity_block(x, 3, [f, f, f * 2], stage=2, block='c')

    x = conv_block(x2, 3, [f * 2, f * 2, f * 4], stage=3, block='a')
    x = identity_block(x, 3, [f * 2, f * 2, f * 4], stage=3, block='b')
    x3 = identity_block(x, 3, [f * 2, f * 2, f * 4], stage=3, block='d')

    x = conv_block(x3, 3, [f * 4, f * 4, f * 8], stage=4, block='a')
    x = identity_block(x, 3, [f * 4, f * 4, f * 8], stage=4, block='b')
    x4 = identity_block(x, 3, [f * 4, f * 4, f * 8], stage=4, block='f')

    x = conv_block(x4, 3, [f * 8, f * 8, f * 16], stage=5, block='a')
    x = identity_block(x, 3, [f * 8, f * 8, f * 16], stage=5, block='b')
    x = identity_block(x, 3, [f * 8, f * 8, f * 16], stage=5, block='c')
    
    
    x = up_conv_block(x, 3, [f * 16, f * 8, f * 8], stage=6, block='a')
    x = identity_block(x, 3, [f * 16, f * 8, f * 8], stage=6, block='b')
    x = identity_block(x, 3, [f * 16, f * 8, f * 8], stage=6, block='c')

    #x = concatenate([x, x4], axis=bn_axis)

    x = up_conv_block(x, 3, [f * 16, f * 4, f * 4], stage=7, block='a')
    x = identity_block(x, 3, [f * 16, f * 4, f * 4], stage=7, block='b')

    x = identity_block(x, 3, [f * 16, f * 4, f * 4], stage=7, block='f')

    #x = concatenate([x, x3], axis=bn_axis)

    x = up_conv_block(x, 3, [f * 8, f * 2, f * 2], stage=8, block='a')
    x = identity_block(x, 3, [f * 8, f * 2, f * 2], stage=8, block='b')
    x = identity_block(x, 3, [f * 8, f * 2, f * 2], stage=8, block='d')

    #x = concatenate([x, x2], axis=bn_axis)

    x = up_conv_block(x, 3, [f * 4, f, f], stage=10, block='a', strides=(1, 1))
    x = identity_block(x, 3, [f * 4, f, f], stage=10, block='b')
    x = identity_block(x, 3, [f * 4, f, f], stage=10, block='c')

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(classes, (3, 3), padding='same', activation='sigmoid', name='convLast')(x)

    model = Model(inp, x, name='resnetUnet')
    #model.compile(optimizer=Adam(lr=3e-4), loss=dice_coef_loss,
                  #metrics=[dice_coef, 'accuracy', precision, recall, f1score])

    #model.summary()

    return model


In [31]:
model = get_resnet()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 720, 960, 3)  0                                            
__________________________________________________________________________________________________
cropping2d_2 (Cropping2D)       (None, 704, 960, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
zero_padding2d_2 (ZeroPadding2D (None, 712, 968, 3)  0           cropping2d_2[0][0]               
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 353, 481, 16) 2368        zero_padding2d_2[0][0]           
__________________________________________________________________________________________________
bn_conv1 (

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# train_generator = train_datagen.flow_from_directory(
#         'data/train',
#         target_size=(150, 150),
#         batch_size=32,
#         color_mode='rgb'        
#         shuffle = True,
#         seed = 42)

# validation_generator = test_datagen.flow_from_directory(
#         'data/validation',
#         target_size=(150, 150),
#         batch_size=32,
#         class_mode='binary')

# model.fit_generator(
#         train_generator,
#         steps_per_epoch=2000,
#         epochs=50,
#         validation_data=validation_generator,
#         validation_steps=800)
# train_generator = train_datagen.flow_from_directory(
#     directory=r"./train/",
#     target_size=(224, 224),
#     color_mode="rgb",
#     batch_size=32,
#     class_mode="categorical",
#     shuffle=True,
#     seed=42
# )

In [ ]:
# data_dir = 'D:/CVLab/Semantic-Segmentation-Suite-master/Semantic-Segmentation-Suite-master/CamVid/'
# train_image_generator = train_datagen.flow_from_directory(
# data_dir + 'train',
# batch_size = 16,
# shuffle = True,
# seed = 42)

# train_mask_generator = train_datagen.flow_from_directory(
# data_dir + 'train_labels',
# batch_size = 16,
# shuffle = True,
# seed = 42)

# val_image_generator = val_datagen.flow_from_directory(
# data_dir + 'val',
# batch_size = 16,
# shuffle = True,
# seed = 52)

# val_mask_generator = val_datagen.flow_from_directory(
# data_dir + 'val_labels',
# batch_size = 16,
# shuffle = True,
# seed = 52)

# test_image_generator = test_datagen.flow_from_directory(
# data_dir + 'test',
# batch_size = 16,
# shuffle = True,
# seed = 62)


# test_mask_generator = test_datagen.flow_from_directory(
# test_labels,
# batch_size = 16,
# shuffle = True,
# seed = 62)



# train_generator = zip(train_image_generator, train_mask_generator)
# val_generator = zip(val_image_generator, val_mask_generator)
# test_generator = zip(test_image_generator, test_mask_generator)

In [32]:
import cv2
import random

def data_gen(img_folder, mask_folder, batch_size):
  c = 0
  img_list = os.listdir(img_folder) #List of training images
  mask_list = os.listdir(mask_folder)  
  n = list(zip(img_list, mask_list))

  random.shuffle(n)

  p, q = zip(*n)

  
  while (True):
    img = np.zeros((batch_size, 720, 960, 3)).astype('float')
    mask = np.zeros((batch_size, 704, 960, 3)).astype('float')

    for i in range(c, c+batch_size): #initially from 0 to 16, c = 0. 

      train_img = cv2.imread(img_folder+'/'+p[i])/255.
      
      
      img[i-c] = train_img #add to array - img[0], img[1], and so on.
                                                   

      train_mask = cv2.imread(mask_folder+'/'+q[i])/255.
      train_mask = train_mask[8:712,:]
      
      mask[i-c] = train_mask

    c+=batch_size
    if(c+batch_size>=len(os.listdir(img_folder))):
      c=0
      random.shuffle(n)
      p, q = zip(*n)
                  # print "randomizing again"
    yield img, mask


In [33]:
train_frame_path = 'D:/CVLab/Semantic-Segmentation-Suite-master/Semantic-Segmentation-Suite-master/CamVid/train'
train_mask_path = 'D:/CVLab/Semantic-Segmentation-Suite-master/Semantic-Segmentation-Suite-master/CamVid/train_labels'

val_frame_path = 'D:/CVLab/Semantic-Segmentation-Suite-master/Semantic-Segmentation-Suite-master/CamVid/val'
val_mask_path = 'D:/CVLab/Semantic-Segmentation-Suite-master/Semantic-Segmentation-Suite-master/CamVid/val_labels'

test_frame_path = 'D:/CVLab/Semantic-Segmentation-Suite-master/Semantic-Segmentation-Suite-master/CamVid/test'
test_mask_path = 'D:/CVLab/Semantic-Segmentation-Suite-master/Semantic-Segmentation-Suite-master/CamVid/test_labels'
# Train the model
train_generator = data_gen(train_frame_path,train_mask_path, batch_size = 8)
val_generator = data_gen(val_frame_path,val_mask_path, batch_size = 8)
test_generator = data_gen(test_frame_path,test_mask_path, batch_size = 8)

In [34]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import CSVLogger
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam



NO_OF_TRAINING_IMAGES = len(os.listdir('D:/CVLab/Semantic-Segmentation-Suite-master/Semantic-Segmentation-Suite-master/CamVid/train/'))
NO_OF_VAL_IMAGES = len(os.listdir('D:/CVLab/Semantic-Segmentation-Suite-master/Semantic-Segmentation-Suite-master/CamVid/val/'))

NO_OF_EPOCHS = 128

BATCH_SIZE = 8

weights_path = 'D:/CVLab/Semantic-Segmentation-Suite-master/Semantic-Segmentation-Suite-master/CamVid/weights'


opt = Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

model.compile(optimizer=Adam(lr=3e-4), loss='mean_squared_error', metrics = ['accuracy'])
                  #metrics=[dice_coef, 'accuracy', precision, recall, f1score])

# checkpoint = ModelCheckpoint(weights_path, monitor='METRIC_TO_MONITOR', 
#                              verbose=1, save_best_only=True, mode='max')

# csv_logger = CSVLogger('./log.out', append=True, separator=';')

# earlystopping = EarlyStopping(monitor = 'METRIC_TO_MONITOR', verbose = 1,
#                               min_delta = 0.01, patience = 3, mode = 'max')

# callbacks_list = [checkpoint, csv_logger, earlystopping]

results = model.fit_generator(train_generator, epochs=NO_OF_EPOCHS, 
                          steps_per_epoch = (NO_OF_TRAINING_IMAGES//BATCH_SIZE),
                          validation_data=val_generator, 
                          validation_steps=(NO_OF_VAL_IMAGES//BATCH_SIZE)) 
                        
#model.save('Model.h5')

Epoch 1/128


ResourceExhaustedError: OOM when allocating tensor with shape[8,176,240,32] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: training_6/Adam/gradients/zeros_69-0-1-TransposeNCHWToNHWC-LayoutOptimizer = Transpose[T=DT_FLOAT, Tperm=DT_INT32, _device="/job:localhost/replica:0/task:0/device:GPU:0"](training_6/Adam/gradients/zeros_69, PermConstNCHWToNHWC-LayoutOptimizer)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: loss_7/mul/_17329 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_29254_loss_7/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_